# Construction of the Keras framework to build recommender systems. 

key ideas :
1) We take advantage of the power of a modern computation framework like Keras (i.e. automatic differentiation) to implement a vanilla matrix factorization recommender with minimal code.
2) We make it a bit more complex by adding users biais and items biais as well as a non linear output function
3) We finally design a neural network

The data used for this task is the smallest MovieLens data set already saved to a local directory so we can get started with some imports and reading in the ratings.csv file, which is where the data for this task comes from.

In [28]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Movie lens dataset

This dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. It contains **100836 ratings** and 3683 tag applications across **9724 movies**. These data were created by **610 users** between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018.

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in the files `links.csv`, `movies.csv`, `ratings.csv` and `tags.csv`. More details about the contents and use of all these files follows.

This is a *development* dataset. As such, it may change over time and is not an appropriate dataset for shared research results. See available *benchmark* datasets if that is your intent.

This and other GroupLens data sets are publicly available for download at <http://grouplens.org/datasets/> **Dowlaod dataset from [here](https://grouplens.org/datasets/movielens/)**

In [29]:
PATH = "./ml-latest-small/"
ratings = pd.read_csv(PATH + "ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


The data is tabular and consists of a user ID, a movie ID, and a rating (there’s also a timestamp but we won’t use it for this task). Our task is to predict the rating for a user/movie pair, with the idea that if we had a model that’s good at this task then we could predict how a user would rate movies they haven’t seen yet and recommend movies with the highest predicted rating.

The zip file also includes a listing of movies and their associated genres. We don’t actually need this for the model but it’s useful to know about.

In [30]:
movies = pd.read_csv(PATH + "movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


To get a better sense of what the data looks like, we can turn it into a table by selecting the top 15 users/movies from the data and joining them together. The result shows how each of the top users rated each of the top movies.

In [31]:
g = ratings.groupby("userId")["rating"].count()
top_users = g.sort_values(ascending=False)[:15]
g = ratings.groupby("movieId")["rating"].count()
top_movies = g.sort_values(ascending=False)[:15]
top_r = ratings.join(top_users, rsuffix="_r", how="inner", on="userId")
top_r = top_r.join(top_movies, rsuffix="_r", how="inner", on="movieId")
pd.crosstab(top_r.userId, top_r.movieId, top_r.rating, aggfunc=np.sum)

movieId,1,50,110,260,296,318,356,480,527,589,593,1196,2571,2858,2959
userId,,,,,,,,,,,,,,,
68,2.5,3.0,2.5,5.0,2.0,3.0,3.5,3.5,4.0,3.5,3.5,5.0,4.5,5.0,2.5
182,4.0,4.5,3.5,3.5,5.0,4.5,5.0,3.5,4.0,2.0,4.5,3.0,5.0,5.0,5.0
249,4.0,4.0,5.0,5.0,4.0,4.5,4.5,4.0,4.5,4.0,4.0,5.0,5.0,4.5,5.0
274,4.0,4.0,4.5,3.0,5.0,4.5,4.5,3.5,4.0,4.5,4.0,4.5,4.0,5.0,5.0
288,4.5,NaN,5.0,5.0,5.0,5.0,5.0,2.0,5.0,4.0,5.0,4.5,3.0,NaN,3.5
307,4.0,4.5,3.5,3.5,4.5,4.5,4.0,3.5,4.5,2.5,4.5,3.0,3.5,4.0,4.0
380,5.0,4.0,4.0,5.0,5.0,3.0,5.0,5.0,NaN,5.0,5.0,5.0,4.5,NaN,4.0
387,NaN,4.5,3.5,4.5,5.0,3.5,4.0,3.0,NaN,3.5,4.0,4.5,4.0,4.5,4.5
414,4.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0,4.0,5.0,5.0,5.0,5.0


To build our first collaborative filtering model, we need to take care of a few things first. The user/movie fields are currently non-sequential integers representing some unique ID for that entity. We need them to be sequential starting at zero to use for modeling (you’ll see why later). We can use scikit-learn’s LabelEncoder class to transform the fields. We’ll also create variables with the total number of unique users and movies in the data, as well as the min and max ratings present in the data, for reasons that will become apparent shortly.

In [7]:
user_enc = LabelEncoder()
ratings["user"] = user_enc.fit_transform(ratings["userId"].values)
n_users = ratings["user"].nunique()
item_enc = LabelEncoder()
ratings["movie"] = item_enc.fit_transform(ratings["movieId"].values)
n_movies = ratings["movie"].nunique()
ratings["rating"] = ratings["rating"].values.astype(np.float32)
min_rating = min(ratings["rating"])
max_rating = max(ratings["rating"])
n_users, n_movies, min_rating, max_rating

(610, 9724, 0.5, 5.0)

Create a traditional (X, y) pairing of data and label, then split the data into training and test data sets.

In [8]:
X = ratings[["user", "movie"]].values
y = ratings["rating"].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((90752, 2), (10084, 2), (90752,), (10084,))

Another constant we’ll need for the model is the number of latent factors per user/movie. This number can be whatever we want, however 50 seemed to work best so we’ll go with that.
Finally, we need to turn users and movies into separate arrays in the training and test data. This is because in Keras they’ll each be defined as distinct inputs, and the way Keras works is each input needs to be fed in as its own array.

In [32]:
n_factors = 50
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

# Construct a matrix factorization recommendation system with keras

Now we get to a first model which is the vanilla matrix factorization model. The main idea here is we’re going to use embeddings to represent each user and each movie in the dataset. These embeddings will be vectors (of size n_factors=50) that start out as random numbers but are fit by the model to capture the essential qualities of each user/movie. We can accomplish this by computing the dot product between a user vector and a movie vector to get a predicted rating. The code is fairly simple, there isn’t even a traditional neural network layer or activation involved. I stuck some regularization on the embedding layers and used a different initializer but even that probably isn’t necessary. Notice that this is where we need the number of unique users and movies, since those are required to define the size of each embedding matrix.

In [33]:
from keras.models import Model
from keras.layers import Input, Reshape, Dot
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.regularizers import l2


def RecommenderV1(n_users, n_movies, n_factors):
    user = Input(shape=(1,))  # One hot encoding into a vector of size 610 (nb of users)
    u = Embedding(
        n_users,
        n_factors,
        embeddings_initializer="he_normal",
        embeddings_regularizer=l2(1e-6),
    )(
        user
    )  # Embedding 610->50 with a l2 regularization
    u = Reshape((n_factors,))(u)

    movie = Input(
        shape=(1,)
    )  # One hot encoding into a vector of size 9724 (nb of movies)
    m = Embedding(
        n_movies,
        n_factors,
        embeddings_initializer="he_normal",
        embeddings_regularizer=l2(1e-6),
    )(
        movie
    )  # Embedding 9724->50 with a l2 regularization
    m = Reshape((n_factors,))(m)

    x = Dot(axes=1)([u, m])
    model = Model(inputs=[user, movie], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss="mean_squared_error", optimizer=opt)
    return model

This is kind of a neat example of how flexible and powerful modern computation frameworks like Keras and PyTorch are. Even though these are billed as deep learning libraries, they have the building blocks to **quickly create any computation graph you want and get automatic differentiation essentially for free**. Below you can see that all of the parameters are in the embedding layers, we don’t have any traditional neural net components at all.

In [34]:
model = RecommenderV1(n_users, n_movies, n_factors)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 1, 50)        30500       input_9[0][0]                    
__________________________________________________________________________________________________
embedding_14 (Embedding)        (None, 1, 50)        486200      input_10[0][0]                   
__________________________________________________________________________________________________
reshape_13

We recover the number of parameters to learn for the two embeddings layers via **610(nb of users)*50(dimension of the latent space)=30500** and **9724(nb of movies)*50=486200**

Let’s go ahead and train this for a few epochs and see what we get.


In [35]:
history = model.fit(
    x=X_train_array,
    y=y_train,
    batch_size=64,
    epochs=5,
    verbose=1,
    validation_data=(X_test_array, y_test),
)

Train on 90752 samples, validate on 10084 samples
Epoch 1/5
90752/90752 [==============================] - 9s 98us/step - loss: 9.3618 - val_loss: 3.1964
Epoch 2/5
90752/90752 [==============================] - 8s 93us/step - loss: 1.9611 - val_loss: 1.6230
Epoch 3/5
90752/90752 [==============================] - 9s 94us/step - loss: 1.1066 - val_loss: 1.3503
Epoch 4/5
90752/90752 [==============================] - 8s 93us/step - loss: 0.8477 - val_loss: 1.2677
Epoch 5/5
90752/90752 [==============================] - 8s 93us/step - loss: 0.7186 - val_loss: 1.2370


# Add a bias and a sigmoid output
Not bad for a first try. We can make some improvements though. 

The first thing we can do is add a “bias” to each embedding. The concept is similar to the bias in a fully-connected layer or the intercept in a linear model. It just provides an extra degree of freedom. We can implement this idea using new embedding layers with a vector length of one. The bias embeddings get added to the result of the dot product.

The second improvement we can make is running the output of the dot product through a sigmoid layer and then scaling the result using the min and max ratings in the data. This is a neat technique that introduces a non-linearity into the output and results in a modest performance bump.

I also refactored the code a bit by pulling out the embedding layer and reshape operation into a separate class.

In [14]:
from keras.layers import Add, Activation, Lambda


class EmbeddingLayer:
    def __init__(self, n_items, n_factors):
        self.n_items = n_items
        self.n_factors = n_factors

    def __call__(self, x):
        x = Embedding(
            self.n_items,
            self.n_factors,
            embeddings_initializer="he_normal",
            embeddings_regularizer=l2(1e-6),
        )(x)
        x = Reshape((self.n_factors,))(x)
        return x


def RecommenderV2(n_users, n_movies, n_factors, min_rating, max_rating):
    user = Input(shape=(1,))
    u = EmbeddingLayer(n_users, n_factors)(user)
    ub = EmbeddingLayer(n_users, 1)(user)  # It is the bias term for users

    movie = Input(shape=(1,))
    m = EmbeddingLayer(n_movies, n_factors)(movie)
    mb = EmbeddingLayer(n_movies, 1)(movie)  # It is the bias term for movies
    x = Dot(axes=1)([u, m])
    x = Add()([x, ub, mb])
    x = Activation("sigmoid")(x)  # The model is sigma(<u,m> + ub + um)
    x = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(
        x
    )  # rescale function from [0,1] to the range of the grades
    model = Model(inputs=[user, movie], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss="mean_squared_error", optimizer=opt)
    return model

The model summary shows the new graph. Notice the additional embedding layers with parameter numbers equal to the unique user and movie counts.

In [15]:
model = RecommenderV2(n_users, n_movies, n_factors, min_rating, max_rating)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1, 50)        30500       input_3[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 1, 50)        486200      input_4[0][0]                    
__________________________________________________________________________________________________
reshape_3 

The model summary shows the new graph. Notice the additional embedding layers with parameter numbers equal to the unique user and movie counts (We see here that 610 parameters and 9724 parameters have been added to the model : one bias for each user and each movie.).

In [22]:
history = model.fit(
    x=X_train_array,
    y=y_train,
    batch_size=64,
    epochs=5,
    verbose=1,
    validation_data=(X_test_array, y_test),
)

Train on 90752 samples, validate on 10084 samples
Epoch 1/5
90752/90752 [==============================] - 9s 100us/step - loss: 1.1964 - val_loss: 0.8559
Epoch 2/5
90752/90752 [==============================] - 9s 95us/step - loss: 0.6987 - val_loss: 0.7564
Epoch 3/5
90752/90752 [==============================] - 9s 97us/step - loss: 0.5301 - val_loss: 0.7333
Epoch 4/5
90752/90752 [==============================] - 9s 97us/step - loss: 0.4017 - val_loss: 0.7358
Epoch 5/5
90752/90752 [==============================] - 9s 97us/step - loss: 0.3092 - val_loss: 0.7500


Those two additions to the model resulted in a pretty sizable improvement. Validation error is now down to ~0.76 which is about as good as the SOTA for this data set.
That pretty much covers the conventional approach to solving this problem, but there’s another way we can tackle this. 

# A Recommendation system neural network

Instead of taking the dot product of the embedding vectors, what if we just concatenated the embeddings together and stuck a fully-connected layer on top of them? It’s still not technically “deep” but it would at least be a neural network! To modify the code, we can remove the bias embeddings from V2 and do a concat on the embedding layers instead of the dot product. Then we can add some dropout, insert a dense layer, and stick some dropout on the dense layer as well. Finally, we’ll run it through a single-unit dense layer to keep the sigmoid trick at the end.

In [36]:
from keras.layers import Concatenate, Dense, Dropout


def RecommenderNet(n_users, n_movies, n_factors, min_rating, max_rating):
    user = Input(shape=(1,))
    u = EmbeddingLayer(n_users, n_factors)(
        user
    )  # embedding 610->50 without regularization

    movie = Input(shape=(1,))
    m = EmbeddingLayer(n_movies, n_factors)(
        movie
    )  # embedding 9724->50 without regularization

    x = Concatenate()([u, m])
    x = Dropout(0.05)(
        x
    )  # 5/100 of the units are desactivated at each pass during learning

    x = Dense(10, kernel_initializer="he_normal")(x)  # A dense layer 100+1(bias)->10
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)

    x = Dense(1, kernel_initializer="he_normal")(x)  # A dense layer 10+1(bias)->1
    x = Activation("sigmoid")(x)
    x = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(x)
    model = Model(inputs=[user, movie], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss="mean_squared_error", optimizer=opt)
    return model

In [37]:
model = RecommenderNet(n_users, n_movies, n_factors, min_rating, max_rating)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_15 (Embedding)        (None, 1, 50)        30500       input_11[0][0]                   
__________________________________________________________________________________________________
embedding_16 (Embedding)        (None, 1, 50)        486200      input_12[0][0]                   
__________________________________________________________________________________________________
reshape_15

Most of the parameters are still in the embedding layers (30.500 + 486.200), but we have some added learning capability from the dense layers (101*10 + 11*1 = 1010+11 = 1021 parameters to learn in the two dense layers).

In [38]:
history = model.fit(
    x=X_train_array,
    y=y_train,
    batch_size=64,
    epochs=5,
    verbose=1,
    validation_data=(X_test_array, y_test),
)

Train on 90752 samples, validate on 10084 samples
Epoch 1/5
90752/90752 [==============================] - 10s 111us/step - loss: 0.9082 - val_loss: 0.7941
Epoch 2/5
90752/90752 [==============================] - 10s 105us/step - loss: 0.7734 - val_loss: 0.7783
Epoch 3/5
90752/90752 [==============================] - 9s 104us/step - loss: 0.7425 - val_loss: 0.7810
Epoch 4/5
90752/90752 [==============================] - 9s 104us/step - loss: 0.7266 - val_loss: 0.7811
Epoch 5/5
90752/90752 [==============================] - 9s 104us/step - loss: 0.7164 - val_loss: 0.7779


# Conclusions :

1) Keras can be used to differentiate almost any function !

2) Without doing any tuning at all we still managed to get a result that’s pretty close to the best performance we saw with the traditional approach. This technique has the added benefit that we can easily incorporate additional features into the model. For instance, we could create some date features from the timestamp or throw in the movie genres as a new embedding layer. We could tune the size of the movie and user embeddings independently since they no longer need to match. Lots of possibilities here.